In [ ]:
!pip install insightface

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 439.5/439.5 kB 4.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.5/17.5 MB 85.9 MB/s eta 0:00:00
  Created wheel for insightface: filename=insightface-0.7.3-cp312-cp312-linux_x86_64.whl size=1071489 sha256=1e84ca7e58de0f3f687359d35cc520082a91a0278a9d9a88b8b9a6f425c13ba4
  Stored in directory: /root/.cache/pip/wheels/73/3c/e2/6d4815e8a8b33a2006554d65ce0d1f973e768f4c7a222fa675
Successfully built insightface


In [ ]:
!pip install onnxruntime

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 44.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 5.8 MB/s eta 0:00:00


In [ ]:
import os
import cv2
import numpy as np
import shutil
import zipfile
import tempfile
import random
import gc
from PIL import Image
from pathlib import Path
from tqdm import tqdm
from insightface.app import FaceAnalysis
from google.colab import drive

In [ ]:
drive.mount('/content/drive')

SEED = 42
random.seed(SEED)
np.random.seed(SEED)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
RAW_TEST_PATH = "/content/drive/MyDrive/HECTO/Dataset/test_data"
SAVE_TEST_PATH = "/content/drive/MyDrive/HECTO/Dataset/test_pp"
ZIP_SAVE_PATH = "/content/drive/MyDrive/HECTO/Dataset/test_pp.zip"

In [ ]:
TARGET_SIZE_PHOTO = (256, 256)  # 사진
TARGET_SIZE_VIDEO = (224, 224)  # 영상
NUM_FRAMES = 15  # 영상 프레임 수

In [ ]:
detector = FaceAnalysis(
    allowed_modules=['detection', 'landmark_2d'],
    providers=['CUDAExecutionProvider']
)
detector.prepare(ctx_id=0, det_size=(640, 640))

Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
model ignore: /root/.insightface/models/buffalo_l/1k3d68.onnx landmark_3d_68
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
model ignore: /root/.insightface/models/buffalo_l/2d106det.onnx landmark_2d_106
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/det_10g.onnx detection [1, 3, '?', '?'] 127.5 128.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
model ignore: /root/.insightface/models/buffalo_l/genderage.onnx genderage
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
model ignore: /root/.insightface/models/buffalo_l/w600k_r50.onnx recognition
set det-size: (640, 640)


In [ ]:
def get_hybrid_crop(image, face_info, target_size, scale):
    try:
        h, w = image.shape[:2]
        bbox = face_info.bbox.astype(int)
        landmarks = getattr(face_info, 'kps', None)

        cx, cy = (bbox[0] + bbox[2]) // 2, (bbox[1] + bbox[3]) // 2

        # 얼굴 정렬 (landmark-based)
        if landmarks is not None and len(landmarks) >= 2:
            left_eye, right_eye = landmarks[0], landmarks[1]
            dy, dx = right_eye[1] - left_eye[1], right_eye[0] - left_eye[0]
            angle = np.degrees(np.arctan2(dy, dx))
            M = cv2.getRotationMatrix2D((float(cx), float(cy)), angle, 1.0)
            image = cv2.warpAffine(image, M, (w, h), flags=cv2.INTER_CUBIC)

        # 정사각 크롭
        side = int(max(bbox[2]-bbox[0], bbox[3]-bbox[1]) * scale)
        half = side // 2
        x1, y1 = cx - half, cy - half
        x2, y2 = cx + half, cy + half

        # 패딩 계산
        px1, py1 = max(0, -x1), max(0, -y1)
        px2, py2 = max(0, x2 - w), max(0, y2 - h)
        x1, y1 = max(0, x1), max(0, y1)
        x2, y2 = min(w, x2), min(h, y2)

        crop = image[y1:y2, x1:x2]

        if px1 or py1 or px2 or py2:
            crop = cv2.copyMakeBorder(crop, py1, py2, px1, px2,
                                      cv2.BORDER_CONSTANT, value=[0,0,0])

        return cv2.resize(crop, target_size)
    except:
        return None

In [ ]:
IMG_EXTS = ['jpg', 'jpeg', 'png', 'jfif']
VID_EXTS = ['mp4', 'mov', 'avi']

In [ ]:
if os.path.exists(SAVE_TEST_PATH):
    shutil.rmtree(SAVE_TEST_PATH)
os.makedirs(SAVE_TEST_PATH, exist_ok=True)

# --- 파일 처리 ---
test_files = sorted(os.listdir(RAW_TEST_PATH))
for file_name in tqdm(test_files, desc="Preprocessing"):
    file_path = os.path.join(RAW_TEST_PATH, file_name)
    ext = file_name.lower().split('.')[-1]

    # ---------- 사진 처리 ----------
    if ext in IMG_EXTS:
        image = None
        try:
            # OpenCV로 읽기
            image = cv2.imread(file_path)
            if image is None or image.size == 0:
                # PIL fallback
                image = Image.open(file_path).convert("RGB")
                image = cv2.cvtColor(np.array(image), cv2.COLOR_RGB2BGR)
        except Exception as e:
            print(f"⚠️ 이미지 읽기 실패: {file_name} → {e}")
            continue

        try:
            faces = detector.get(image)
            if faces:
                face = max(faces, key=lambda x: (x.bbox[2]-x.bbox[0])*(x.bbox[3]-x.bbox[1]))
                crop = get_hybrid_crop(image, face, TARGET_SIZE_PHOTO, scale=1.3)
                if crop is None:
                    bbox = face.bbox.astype(int)
                    crop = image[bbox[1]:bbox[3], bbox[0]:bbox[2]]
                    crop = cv2.resize(crop, TARGET_SIZE_PHOTO)
            else:
                crop = cv2.resize(image, TARGET_SIZE_PHOTO)

            if crop.dtype != np.uint8:
                crop = np.clip(crop, 0, 255).astype(np.uint8)

            save_name = Path(file_name).stem + ".jpg"
            save_path = os.path.join(SAVE_TEST_PATH, save_name)
            cv2.imwrite(save_path, crop)
        except Exception as e:
            print(f"⚠️ 전처리 실패: {file_name} → {e}")

    # ---------- 영상 처리 ----------
    elif ext in VID_EXTS:
        cap = cv2.VideoCapture(file_path)
        total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        if total_frames <= 0:
            cap.release()
            continue

        indices = np.linspace(0, total_frames-1, NUM_FRAMES, dtype=int)
        frames_cache = []
        face_crops = []
        last_face_info = None

        # 영상 폴더 생성
        video_folder = os.path.join(SAVE_TEST_PATH, Path(file_name).stem)
        os.makedirs(video_folder, exist_ok=True)

        for idx in indices:
            cap.set(cv2.CAP_PROP_POS_FRAMES, idx)
            ret, frame = cap.read()
            if not ret or frame is None:
                continue
            frames_cache.append(frame)

            faces = detector.get(frame)
            if faces:
                face = max(faces, key=lambda x: (x.bbox[2]-x.bbox[0])*(x.bbox[3]-x.bbox[1]))
                crop = get_hybrid_crop(frame, face, TARGET_SIZE_VIDEO, scale=1.14)

                # 첫 얼굴 이전 프레임 소급 처리
                if last_face_info is None and len(face_crops) == 0 and len(frames_cache) > 1:
                    for i in range(len(frames_cache)-1):
                        retro_crop = get_hybrid_crop(frames_cache[i], face, TARGET_SIZE_VIDEO, scale=1.14)
                        if retro_crop is not None:
                            face_crops.append(retro_crop)
                        else:
                            face_crops.append(crop)
                last_face_info = face
            elif last_face_info is not None:
                crop = get_hybrid_crop(frame, last_face_info, TARGET_SIZE_VIDEO, scale=1.14)
            else:
                crop = None

            if crop is not None:
                if crop.dtype != np.uint8:
                    crop = np.clip(crop, 0, 255).astype(np.uint8)
                face_crops.append(crop)

        # 후처리: 얼굴 못찾거나 프레임 부족
        if len(face_crops) == 0:
            for i, f in enumerate(frames_cache[:NUM_FRAMES]):
                f_resized = cv2.resize(f, TARGET_SIZE_VIDEO)
                if f_resized.dtype != np.uint8:
                    f_resized = np.clip(f_resized, 0, 255).astype(np.uint8)
                save_file = os.path.join(video_folder, f"frame_{i}.jpg")
                cv2.imwrite(save_file, f_resized)
        else:
            while len(face_crops) < NUM_FRAMES:
                face_crops.append(face_crops[-1])
            for i, crop_img in enumerate(face_crops[:NUM_FRAMES]):
                save_file = os.path.join(video_folder, f"frame_{i}.jpg")
                cv2.imwrite(save_file, crop_img)

        cap.release()
        gc.collect()

Preprocessing: 100%|██████████| 500/500 [58:13<00:00,  6.99s/it]


In [ ]:
import os
from pathlib import Path

SAVE_TEST_PATH = "/content/drive/MyDrive/HECTO/Dataset/test_pp"

IMG_EXTS = ['jpg', 'jpeg', 'png', 'jfif']

total_photos = 0
total_videos = 0
video_frame_counts = {}

for item in sorted(os.listdir(SAVE_TEST_PATH)):
    item_path = os.path.join(SAVE_TEST_PATH, item)

    if os.path.isfile(item_path) and item.lower().endswith('.jpg'):
        # 사진 파일(.jpg) 단일로 존재
        total_photos += 1
    elif os.path.isdir(item_path):
        # 영상 프레임 폴더
        frames = [f for f in os.listdir(item_path) if f.lower().endswith('.jpg')]
        video_frame_counts[item] = len(frames)
        total_videos += 1

print(f"✅ 총 사진 개수: {total_photos}")
print(f"✅ 총 영상 개수: {total_videos}")
print("✅ 영상별 프레임 개수 (15 프레임 기준 체크):")
for video, count in video_frame_counts.items():
    status = "✅ 정상" if count == 15 else f"⚠️ {count}개"
    print(f"  {video}: {count} frames → {status}")


✅ 총 사진 개수: 245
✅ 총 영상 개수: 255
✅ 영상별 프레임 개수 (15 프레임 기준 체크):
  TEST_000: 15 frames → ✅ 정상
  TEST_002: 15 frames → ✅ 정상
  TEST_003: 15 frames → ✅ 정상
  TEST_005: 15 frames → ✅ 정상
  TEST_006: 15 frames → ✅ 정상
  TEST_011: 15 frames → ✅ 정상
  TEST_012: 15 frames → ✅ 정상
  TEST_014: 15 frames → ✅ 정상
  TEST_015: 15 frames → ✅ 정상
  TEST_018: 15 frames → ✅ 정상
  TEST_019: 15 frames → ✅ 정상
  TEST_023: 15 frames → ✅ 정상
  TEST_029: 15 frames → ✅ 정상
  TEST_031: 15 frames → ✅ 정상
  TEST_032: 15 frames → ✅ 정상
  TEST_033: 15 frames → ✅ 정상
  TEST_035: 15 frames → ✅ 정상
  TEST_036: 15 frames → ✅ 정상
  TEST_038: 15 frames → ✅ 정상
  TEST_040: 15 frames → ✅ 정상
  TEST_043: 15 frames → ✅ 정상
  TEST_048: 15 frames → ✅ 정상
  TEST_051: 15 frames → ✅ 정상
  TEST_052: 15 frames → ✅ 정상
  TEST_053: 15 frames → ✅ 정상
  TEST_054: 15 frames → ✅ 정상
  TEST_056: 15 frames → ✅ 정상
  TEST_058: 15 frames → ✅ 정상
  TEST_062: 15 frames → ✅ 정상
  TEST_069: 15 frames → ✅ 정상
  TEST_070: 15 frames → ✅ 정상
  TEST_071: 15 frames → ✅ 정상
  TEST_074: 1

In [ ]:
with zipfile.ZipFile(ZIP_SAVE_PATH, 'w', zipfile.ZIP_DEFLATED) as z:
    for root, dirs, files in os.walk(SAVE_TEST_PATH):
        for file in files:
            file_path = os.path.join(root, file)
            arcname = os.path.relpath(file_path, SAVE_TEST_PATH)
            z.write(file_path, arcname)

print(f"📦 ZIP 저장 완료 → {ZIP_SAVE_PATH}")

📦 ZIP 저장 완료 → /content/drive/MyDrive/HECTO/Dataset/test_pp.zip
